# <center> XGBOOST for a regression problem: House Prices </center>

In [ ]:
from pandas import read_csv
import pandas as pd
train=read_csv('../HousePrices/train.csv', header=0, index_col=0)
val=read_csv('../HousePrices/val.csv', header=0, index_col=0)

## Removing the outliers with IsolationForest

In [ ]:
from sklearn.ensemble import IsolationForest

clf = IsolationForest(max_samples = 100, random_state = 42)
clf.fit(train)
y_noano = clf.predict(train)
y_noano = pd.DataFrame(y_noano, columns = ['Top'])
y_noano[y_noano['Top'] == 1].index.values

train = train.iloc[y_noano[y_noano['Top'] == 1].index.values]
train.reset_index(drop = True, inplace = True)
print("Number of Outliers:", y_noano[y_noano['Top'] == -1].shape[0])
print("Number of rows without outliers:", train.shape[0])

GridSearch to find the best hyperparameters de XGBRegressor.  
Attention dure 4h

In [ ]:
#from sklearn.model_selection import GridSearchCV 
#train_x=train[most_relevant_features]
#train_y=train['SalePrice']
#for tuning parameters
#parameters_for_testing = {
#     'learning_rate':[0.1,0.07],
#     'max_depth':[3,5],
#     'n_estimators':[10000],
#     'reg_alpha':[1e-5, 1e-2,  0.75],
#     'reg_lambda':[1e-5, 1e-2, 0.45],
#     'subsample':[0.6,0.95],
#}

                    
#xgb_model = xgboost.XGBRegressor(learning_rate =0.1, n_estimators=1000, max_depth=5, min_child_weight=1, gamma=0, subsample=0.8, colsample_bytree=0.8, nthread=6, scale_pos_weight=1, seed=27)

#gsearch1 = GridSearchCV(estimator = xgb_model, param_grid = parameters_for_testing, n_jobs=6,iid=False, verbose=10,scoring='neg_mean_absolute_error')
#gsearch1.fit(train_x,train_y)
#print (gsearch1.cv_results_)
#print('best params')
#print (gsearch1.best_params_)
#print('best score')
#print (gsearch1.best_score_)


In [ ]:
import xgboost
model = xgboost.XGBRegressor(colsample_bytree=0.4,
                 objective ='reg:squarederror',
                 gamma=0,                 
                 learning_rate=0.07,
                 max_depth=3,
                 min_child_weight=1.5,
                 n_estimators=10000,                                                                    
                 reg_alpha=0.75,
                 reg_lambda=0.45,
                 subsample=0.6,
                 seed=42) 

In [ ]:
every_column_except_y= [col for col in train.columns if col not in ['SalePrice','Id']]
model.fit(train[every_column_except_y],train['SalePrice'])

In [ ]:
every_valcolumn_except_y= [col for col in val.columns if col not in ['SalePrice','Id']]
y_val_predict = model.predict(val[every_valcolumn_except_y])
print(y_val_predict)

## Compute MAE

In [ ]:
from sklearn.metrics import mean_absolute_error
y_val = val['SalePrice']
mean_absolute_error(y_val, y_val_predict)

## Find the most relevant features

In [ ]:
from collections import OrderedDict
feat_imp = OrderedDict(sorted(model.get_booster().get_fscore().items(), key=lambda t: t[1], reverse=True))
feat_imp

In [ ]:
#Let's remove the less important ones 
most_relevant_features= list(dict((k, v) for k, v in model.get_booster().get_fscore().items() if v >= 10).keys())
print(most_relevant_features)

In [ ]:
import matplotlib.pyplot as plt
from xgboost import plot_importance
%matplotlib notebook
plot_importance(model, max_num_features=10) # top 10 most important features
plt.show()

In [ ]:
plot_importance(model)
plt.draw()

## Verify with graph the most relevant features

In [ ]:
#pip install seaborn

In [ ]:
import seaborn as sns
import numpy as np
train['logSalePrice'] = np.log(train['SalePrice'])
graphe1 = sns.jointplot('GrLivArea', 'logSalePrice', data=train, kind="reg")

In [ ]:
graphe = sns.jointplot('LotArea', 'logSalePrice', data=train, kind="reg")

In [ ]:
#removing outliers
train_outliers = train[train.GrLivArea < 4000]
train_outliers = train[train.LotArea < 100000]

In [ ]:
graphe2 = sns.jointplot('GrLivArea', 'logSalePrice', data=train_outliers, kind="reg")

In [ ]:
graphe2 = sns.jointplot('LotArea', 'logSalePrice', data=train_outliers, kind="reg")

## Best model with most relevant features

In [ ]:
X_train=train[most_relevant_features]
y_train=train['SalePrice']

In [ ]:
best_xgb_model = xgboost.XGBRegressor(colsample_bytree=0.4,
                 objective ='reg:squarederror',
                 gamma=0,                 
                 learning_rate=0.07,
                 max_depth=3,
                 min_child_weight=1.5,
                 n_estimators=10000,                                                                    
                 reg_alpha=0.75,
                 reg_lambda=0.45,
                 subsample=0.6,
                 seed=42)

In [ ]:
best_xgb_model.fit(train[most_relevant_features],train['SalePrice'])

## Learning Curve

In [ ]:
from sklearn.model_selection import learning_curve
import numpy as np
from xgboost import XGBRegressor

train_sizes, train_scores, test_scores = learning_curve(XGBRegressor(colsample_bytree=0.4,
                                                     objective ='reg:squarederror',
                                                     gamma=0,                 
                                                     learning_rate=0.07,
                                                     max_depth=3,
                                                     min_child_weight=1.5,
                                                     n_estimators=10000,                                                                    
                                                     reg_alpha=0.75,
                                                     reg_lambda=0.45,
                                                     subsample=0.6,
                                                     seed=42), 
                                                        X_train, 
                                                        y_train_log_scaled,
                                                        # Number of folds in cross-validation
                                                        cv=10,
                                                        # Evaluation metric
                                                        scoring='neg_mean_absolute_error',
                                                        # Use all computer cores
                                                        n_jobs=-1, 
                                                        # 10 different sizes of the training set
                                                        train_sizes=np.linspace(0.01, 1.0, 10))
# Create means and standard deviations of training set scores
train_mean = np.mean(train_scores, axis=1)
train_std = np.std(train_scores, axis=1)

# Create means and standard deviations of test set scores
test_mean = np.mean(test_scores, axis=1)
test_std = np.std(test_scores, axis=1)

# Draw lines
plt.plot(train_sizes, train_mean, '--', color='navy',  label="Training score")
plt.plot(train_sizes, test_mean, color='red', label="Cross-validation score")

# Draw bands
plt.fill_between(train_sizes, train_mean - train_std, train_mean + train_std, color="grey")
plt.fill_between(train_sizes, test_mean - test_std, test_mean + test_std, color="grey")

# Create plot
plt.title("Learning Curve")
plt.xlabel("Training Set Size"), plt.ylabel("Score"), plt.legend(loc="best")
plt.tight_layout()
plt.show()

## Compute MAE

In [ ]:
y_val_predict = best_xgb_model.predict(val[most_relevant_features])
y_val = val['SalePrice']
mean_absolute_error(y_val, y_val_predict)

## Prediction on Test

In [ ]:
test = read_csv('../HousePrices/test.csv', header=0)
test

Modify test like trainfull

In [ ]:
test_object = test.select_dtypes(include='object')
test_object = test_object.fillna('None')

In [ ]:
test_object = pd.get_dummies(test_object)
test_object = test_object.astype(int)
test_object = test_object.drop(['MSZoning_RL'], axis=1)

In [ ]:
test_number = test.select_dtypes(exclude='object')
test_number = test_number.fillna(0)

In [ ]:
import pandas as pd
test = pd.concat([test_number, test_object], axis=1)
test

Make prediction

In [ ]:
test['Prediction'] = best_xgb_model.predict(test[most_relevant_features])

In [ ]:
filename = 'prediction_XGBOOST.csv'
pd.DataFrame({'Id': test.Id, 'SalePrice': test.Prediction}).to_csv(filename, index=False)

In [ ]:
print(test['Prediction'].head())
print(test['Prediction'].count())

In [ ]:
test.to_csv("test_modified.csv")

# Score Kaggle
<center> Sans Standardiser les données </center>
Best_Xgb_Model : 0.13504  
<center> Avec Standardisation </center>